# Support Vector Machine

Este notebook apresenta a utilização da SVM para a classificação para classificar os sensores de detecção da bactéria E-Coli, desenvolvidos pelo grupo de pesquisa do Programa de Pós-Graduação em Física da UFMG, conforme sua etapa de fabricação, dado uma curva de tensão de entrada e suas respectivas medidas de tensão de saída.

## Upload do dataset tratado previamente para o servidor

In [5]:
from google.colab import files
uploaded = files.upload()

Saving all_treated.csv to all_treated.csv


## Importação das bibliotecas necessárias

In [0]:
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn import svm
from sklearn.model_selection import train_test_split, GridSearchCV
import io

## Leitura do dataset

In [0]:
df = pd.read_csv(io.BytesIO(uploaded['all_treated.csv']), sep=';')

X = []
Y = []

for label in df['label'].unique():
    for sample in df['amostra'].unique():
        x = np.reshape(
            df.loc[
                (df['label'] == label) & (df['amostra'] == sample),
                ['v_gate', 'resist']
            ].values,
            -1
        )
        y = label

        if len(x) == 140:
            X.append(x)
            Y.append(y)

X = np.vstack(X)
Y = np.array(Y)

## Treinamento e apuração das métricas de um classificador com configuração padrão

In [8]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)
clf = svm.SVC(gamma='scale')
clf.fit(x_train, y_train)

y_true, y_pred = y_test, clf.predict(x_test)
target_names = ['Limpo', 'Tionina', 'SA20']
print(metrics.accuracy_score(y_true, y_pred))
print(metrics.classification_report(y_true, y_pred, target_names=target_names))

0.6379310344827587
              precision    recall  f1-score   support

       Limpo       0.80      0.77      0.79        57
     Tionina       0.68      0.39      0.49        59
        SA20       0.52      0.76      0.62        58

    accuracy                           0.64       174
   macro avg       0.66      0.64      0.63       174
weighted avg       0.66      0.64      0.63       174



## Busca por hiperparâmetros que melhorem as métricas de qualidade do classificador e melhores resultados encontrados

Hiperparâmetros são os parâmetros passados quando da criação de uma instância de um classificador. Os hiperparâmetros determinam características/comportamentos do classificador instanciado. As possibilidades de configuração da SVM utilizada consta na [documentção](https://scikit-learn.org/stable/modules/svm.html) do fabricante. Neste caso, há interesse principalmente em encontrar o melhor kernel.

O Kernel está associado aos dados do problema (linearmente separáveis ou não linearmente separáveis). São funções usadas para mapear os dados em um espaço de maior dimensão, conforme brevemente apresentado na sessão 3.1.2. `Degree` e `coef0` são parâmetros dependentes do kernel que determinal o grau do polinômio a ser utilizado.

A função `GridSearchCV` utilizada é responsável por instanciar vários classificadores conforme hiperparâmetros/configurações definidas em `parameters` e econtrar o melhor conjunto de hiperparâmetros. Os resultados são mostrados a seguir.

In [ ]:
parameters = [
    {
        'kernel': ['linear'],
        'decision_function_shape': ['ovo', 'ovr'],
    },
    {
        'kernel': ['poly'],
        'decision_function_shape': ['ovo', 'ovr'],
        'gamma': ['auto', 'scale'],
        'degree': [x for x in range(2, 3, 1)],
        'coef0': [x for x in range(1, 3, 1)],
    },
    {
        'kernel': ['sigmoid'],
        'decision_function_shape': ['ovo', 'ovr'],
        'gamma': ['auto', 'scale'],
        'coef0': [x for x in range(1, 3, 1)],
    },
    {
        'kernel': ['rbf'],
        'decision_function_shape': ['ovo', 'ovr'],
        'gamma': ['auto', 'scale'],
        'coef0': [x for x in range(1, 3, 1)],
    },
]

svc = svm.SVC(gamma='scale')
clf = GridSearchCV(svc, parameters, verbose=True, n_jobs=-1, cv=5)
clf.fit(x_train, y_train)

print(clf.best_params_)

In [9]:
y_true, y_pred = y_test, clf.predict(x_test)
print(metrics.accuracy_score(y_true, y_pred))
print(metrics.classification_report(y_true, y_pred, target_names=target_names))

Fitting 5 folds for each of 26 candidates, totalling 130 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed: 385.4min
[Parallel(n_jobs=-1)]: Done 130 out of 130 | elapsed: 387.6min finished
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


{'coef0': 1, 'decision_function_shape': 'ovo', 'degree': 2, 'gamma': 'auto', 'kernel': 'poly'}
0.8850574712643678
              precision    recall  f1-score   support

       Limpo       0.98      0.96      0.97        57
     Tionina       0.87      0.80      0.83        59
        SA20       0.81      0.90      0.85        58

    accuracy                           0.89       174
   macro avg       0.89      0.89      0.89       174
weighted avg       0.89      0.89      0.89       174

